In [3]:

from sklearn.feature_selection import RFE
from sklearn.svm import SVR
import matplotlib.pyplot as plt
import pandas as pd
import time
import numpy as np
from sklearn.cluster import KMeans
from sklearn.svm import SVR
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import RFECV
from sklearn.tree import DecisionTreeClassifier
start_time = time.time()
print(start_time)

1687860623.4973943


In [4]:
#Set labels
labels = ["Labeled_miRNA.csv", "Labeled_methyl.csv","Labeled_gene.csv",]

In [5]:
#Feature selection method for RFECV

def feature_selection(datasetName, rs):
    #Get starting time
    start_time = time.time()
    
    #Set inputpath
    inputPath = "C:/Users/matth/anaconda3/envs/data1/data1/brca/Labeled_data/"
    dataset = inputPath + datasetName
    
    #Read in data as dataframe
    data = pd.read_csv(dataset)
    data = pd.DataFrame(data)
    
    #Conversion between R and Python with reticulate package causes some weird interactions. 
    #To fix these interactions, the datasets have to be loaded in this way
    data.columns = data.iloc[0]
    data = data[1:]
    data = data.apply(pd.to_numeric, errors='ignore')
    
    #Set Ground labels and features
    Y = data['class']
    X = data.drop(['class'], axis=1)
    
    #Train and fit the model
    XTrain, XTest, YTrain, YTest = train_test_split(X, Y, test_size=0.2, random_state = rs)
    model = DecisionTreeClassifier()
    rfecv = RFECV(estimator=model, step=1, cv=5, scoring="accuracy", n_jobs = -1)
    rfecv = rfecv.fit(XTrain, YTrain)
    
    #Get the selected features
    i = 0
    features = []
    for feature in rfecv.support_:
        if feature == False:
            features.append(rfecv.feature_names_in_[i])
        i += 1
        
    #Augment data with desired features
    dropped_data = data.drop(features, axis=1)
    dropped_data = dropped_data.drop(['class'], axis=1)
    
    #Write CSV for later use
    outputPath = "C:/Users/matth/anaconda3/envs/data1/data1/brca/fs_brca/BRCA_FS_RFW_" + datasetName[:-4] + "_" + str(rs) + ".csv"
    dropped_data.to_csv(outputPath)
    end_time = time.time() - start_time
    print("It took {0:2f} secondes for {1} with random state {2} to complete".format(end_time, datasetName, rs))
    

In [7]:
#Run RFECV for datasets
for i in range(1, 2):
    for label in labels:
        feature_selection(label, i)
        
